In [1]:
import os
os.environ["WANDB_DISABLED"] = "true"

In [2]:
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
import torch
print("Pytorch version：")
print(torch.__version__)
print("CUDA Version: ")
print(torch.version.cuda)
print("cuDNN version is :")
print(torch.backends.cudnn.version())

Pytorch version：
2.5.0+cu124
CUDA Version: 
12.4
cuDNN version is :
90100


In [19]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 10240 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Math-7B-bnb-4bit", # "unsloth/Meta-Llama-3.1-8B"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

==((====))==  Unsloth 2024.10.0: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.448 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.6. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


In [20]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

# Classifier

In [6]:
alpaca_prompt = """As a mathematical assistant, You need to analyze the problem to find out what type of problem it belongs to in Real Analysis. Provide the Problem_Type and the Knowledges which may be used to solve this problem.

### Problem:
{}

### Problem_Type:
{}

### Knowledge:
{}"""

In [7]:
EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    Problem_Type    = examples["ProblemType"]
    Problem       = examples["Problem"]
    Knowledge  = examples["Knowledge"]
    texts = []
    for problem, problem_type, knowledge in zip(Problem, Problem_Type, Knowledge):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(problem, problem_type, knowledge) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset('csv',data_files = './pretraining_data_with_knowledge.csv', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 49/49 [00:00<00:00, 5511.27 examples/s]


In [10]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset, # dataset -> tokenized_train_dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 300, # 60 -> 10

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
Map (num_proc=2): 100%|██████████| 49/49 [00:00<00:00, 104.51 examples/s]
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


In [11]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A40. Max memory = 44.448 GB.
5.471 GB of memory reserved.


In [12]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 49 | Num Epochs = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 300
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,1.753600
2,1.755100
3,1.768500
4,1.729200
5,1.764700
6,1.611700
7,1.411300
8,1.281400
9,1.156800
10,1.089400


In [13]:
model.save_pretrained("MathAnalysis_Qwen_Classifier") # Local saving
tokenizer.save_pretrained("MathAnalysis_Qwen_Classifier")

('MathAnalysis_Qwen_Classifier/tokenizer_config.json',
 'MathAnalysis_Qwen_Classifier/special_tokens_map.json',
 'MathAnalysis_Qwen_Classifier/vocab.json',
 'MathAnalysis_Qwen_Classifier/merges.txt',
 'MathAnalysis_Qwen_Classifier/added_tokens.json',
 'MathAnalysis_Qwen_Classifier/tokenizer.json')

In [14]:
test_prompt = """As a mathematical assistant, You need to analyze the problem to find out what type of problem it belongs to in Real Analysis. Provide the Problem_Type and the Knowledges which may be used to solve this problem.

### Problem:
{}
"""

In [15]:
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "MathAnalysis_Qwen_Classifier", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!

inputs = tokenizer(
[
    test_prompt.format(
        "Prove that $n \sin(2n! e \pi)$ converges to $2\pi$ as $n \to \infty$.", # Problem
    )
], return_tensors = "pt").to("cuda")


==((====))==  Unsloth 2024.10.0: Fast Qwen2 patching. Transformers = 4.44.2.
   \\   /|    GPU: NVIDIA A40. Max memory: 44.448 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.5.0+cu124. CUDA = 8.6. CUDA Toolkit = 12.4.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.28.post1. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Unsloth: unsloth/Qwen2.5-Math-7B-bnb-4bit can only handle sequence lengths of at most 4096.
But with kaiokendev's RoPE scaling of 2.5, it can be magically be extended to 10240!


In [16]:
from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(input_ids = inputs.input_ids, attention_mask = inputs.attention_mask,
                   streamer = text_streamer, max_new_tokens = max_seq_length, pad_token_id = tokenizer.eos_token_id)

Reference: T. J. R. Higgins, On the irrationality of $e$, Amer. Math. Monthly 777 (19664) 41933--4994.

### Problem_Type:
Sequences and Limits

### Knowledge:
\begin{itemize}
    \item Let $\{a_n\}$ be a sequence of real or complex numbers. A necessary and sufficient condition for the sequence to converge is that for any $\epsilon > 0$ there exists an integer $N > 0$ such that
    \[
    |a_p - a_q| < \epsilon
    \]
   holds for all integers $p$ and $q$ greater than $N$. This is called the \textit{Cauchy criterion}.
    
    \item Any monotone bounded sequence is convergent.
    
    \item For any sequence $\{a_n\}$, the \textit{inferior limit} and the \textit{superior limit} are defined by the limits of of monotone sequences:
    \[
    \liminf_{n \to \infty} a_n = \lim_{n \to \infty} \inf\{a_n, a_{n+1}, \dots\}
    \]
    and
    \[
       \limsup_{n \to \infty} a_n = \lim_{n \to \infty} \sup\{a_n, a_{n+1}, \dots\}
    \]
    respectively. Note that the inferior and superior limits 

# Problem Solver

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 10240 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3-mini-4k-instruct",          # Phi-3 2x faster!d
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen2.5-Math-7B-bnb-4bit", # "unsloth/Meta-Llama-3.1-8B"
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [21]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Already have LoRA adapters! We shall skip this step.


In [22]:
alpaca_prompt = """As a mathematical assistant, solve the following problem. Provide a detailed, step-by-step solution using rigorous mathematical reasoning. If the problem requires the use of the $\epsilon$-$\delta$ method (e.g., when proving limits or continuity), ensure that you apply it appropriately. Use precise mathematical language and notation throughout your solution.

### Problem_Type:
{}

### Problem:
{}

### Knowledge:
{}

### Solution:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    Problem_Type    = examples["ProblemType"]
    Problem       = examples["Problem"]
    Knowledge = examples["Knowledge"]
    Solution  = examples["Solution"]
    texts = []
    for problem_type, problem, knowledge, solution in zip(Problem_Type, Problem, Knowledge, Solution):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(problem_type, problem, knowledge, solution) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

from datasets import load_dataset
dataset = load_dataset('csv',data_files = './pretraining_data_with_knowledge.csv', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

In [23]:
len(dataset)

49

In [24]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset, # dataset -> tokenized_train_dataset
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        
        # Use num_train_epochs = 1, warmup_ratio for full training runs!
        warmup_steps = 5,
        max_steps = 300, # 60 -> 10

        learning_rate = 2e-4,
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Map (num_proc=2): 100%|██████████| 49/49 [00:00<00:00, 115.03 examples/s]
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
max_steps is given, it will override any value given in num_train_epochs


In [25]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA A40. Max memory = 44.448 GB.
16.451 GB of memory reserved.


In [26]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 49 | Num Epochs = 50
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 300
 "-____-"     Number of trainable parameters = 40,370,176


Step,Training Loss
1,1.705700
2,1.746700
3,1.708600
4,1.718300
5,1.694900
6,1.591100
7,1.404000
8,1.251100
9,1.152600
10,1.032200


In [27]:
model.save_pretrained("MathAnalysis_Qwen_ProblemSolver") # Local saving
tokenizer.save_pretrained("MathAnalysis_Qwen_ProblemSolver")
# model.push_to_hub("your_name/lora_model", token = "...") # Online saving
# tokenizer.push_to_hub("your_name/lora_model", token = "...") # Online saving

('MathAnalysis_Qwen_ProblemSolver/tokenizer_config.json',
 'MathAnalysis_Qwen_ProblemSolver/special_tokens_map.json',
 'MathAnalysis_Qwen_ProblemSolver/vocab.json',
 'MathAnalysis_Qwen_ProblemSolver/merges.txt',
 'MathAnalysis_Qwen_ProblemSolver/added_tokens.json',
 'MathAnalysis_Qwen_ProblemSolver/tokenizer.json')